In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random

from connect4.connect_state import ConnectState
from connect4.policy import Policy

from policy import SmartMCTS

from groups.GroupA.policy import Aha
from groups.GroupB.policy import Hello
from groups.GroupC.policy import OhYes


In [5]:
def play_game(policy_red: Policy, policy_yellow: Policy, render=False):
    state = ConnectState()
    policies = {-1: policy_red, 1: policy_yellow}

    while not state.is_final():
        p = state.player
        action = policies[p].act(state.board)

        if not state.is_applicable(action):
            # Illegal move loses
            return -p
        
        state = state.transition(action)
        
        if render:
            print(state.board)
            print("---------------")

    return state.get_winner()


In [6]:
def evaluate(red_policy_class, yellow_policy_class, n_games=100):
    results = { "red": 0, "yellow": 0, "draw": 0 }

    for i in range(n_games):
        red = red_policy_class()
        yellow = yellow_policy_class()
        red.mount()
        yellow.mount()

        w = play_game(red, yellow)
        if w == -1:
            results["red"] += 1
        elif w == 1:
            results["yellow"] += 1
        else:
            results["draw"] += 1

    return results


In [7]:
class ClaimevenPolicy(Policy):
    def mount(self): pass

    def act(self, board):
        legal = [c for c in range(7) if board[0,c] == 0]

        # 1. Jugar casillas pares (0,2,4,6)
        even_cols = [c for c in legal if c % 2 == 0]
        if even_cols:
            return random.choice(even_cols)

        return random.choice(legal)


In [8]:
class AllisStrategyPolicy(Policy):
    def mount(self): pass

    def get_legal(self, board):
        return [c for c in range(7) if board[0,c] == 0]

    def is_winning_move(self, board, player, col):
        tmp = board.copy()
        for r in range(5, -1, -1):
            if tmp[r,col] == 0:
                tmp[r,col] = player
                break
        return SmartMCTS._get_winner(self, tmp) == player

    def act(self, board):
        legal = self.get_legal(board)
        player = -1 if np.sum(board!=0) % 2 == 0 else 1
        opp = -player

        # 1. Ganar si es posible
        for c in legal:
            if self.is_winning_move(board, player, c):
                return c

        # 2. Bloquear victoria inmediata
        for c in legal:
            if self.is_winning_move(board, opp, c):
                return c

        # 3. Crear amenazas impares
        odd_cols = [c for c in legal if c % 2 == 1]
        if odd_cols:
            return random.choice(odd_cols)

        # 4. Follow-up en misma columna si hay 3 en línea
        for c in legal:
            if self.is_potential_threat(board, player, c):
                return c
        
        # 5. fallback
        return random.choice(legal)

    def is_potential_threat(self, board, player, col):
        return False  # puedes expandirlo si quieres
